In [1]:
import itertools
import mne
import os
import random
import re
import shutil
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scikitplot as skplt
import scipy.io as io
import tensorflow as tf


from datetime import datetime
from keras.wrappers.scikit_learn import KerasClassifier
from pywt import wavedec
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from tensorflow import keras
from typing import List



2022-05-22 22:02:25.349130: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-05-22 22:02:25.473755: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-22 22:02:25.473772: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
filename = "resources/dane_eeg.zip"
extract_dir = "resources/dane_eeg"
seizure_occ_file = "resources/dane_eeg/czasy_napadow.m"
healthy_dir = "resources/dane_eeg/zdrowi"
unhealthy_dir = "resources/dane_eeg/chorzy"
img_dir = "resources/dane_eeg/images/"
gd_path = "resources/gdsearch/"
# shutil.unpack_archive(filename, extract_dir)

In [3]:
def get_mat_files(base_path): 
    mat_files_list = []
    filenames = {}

    for file in os.listdir(base_path):
        if file.endswith(".mat"):
            mat_files_list.append(base_path + "/" + file)
            
    for filename in mat_files_list:        
        filenames[filename.split('/')[-1].split('_')[0]] = filename
        
    return mat_files_list, filenames


healthy_paths, healthy_patients_dict = get_mat_files(healthy_dir)
unhealthy_paths, unhealthy_patients_dict = get_mat_files(unhealthy_dir)

In [4]:
def get_all_seizure_occurrences(file_path):
    all_occurrences = {}
    with open (file_path , 'r') as file:
        for line in file :
            line_number = int(re.search(r'{([0-9]+)}', line).group(1))
            occ_list = list(map(int, re.search(r'\[(.+?)]', line).group(1).split()))
            all_occurrences[line_number] = occ_list
    return all_occurrences


seizure_occurrences = get_all_seizure_occurrences(seizure_occ_file)

In [5]:
class Seizure:
    def __init__ (self, id, filename, samples, seizure_occurrences=None, frequency=100, channels=16) -> None :
        super (). __init__ ()
        self.id = id
        self.filename = filename
        self.samples = samples
        self.seizure_occurrences = seizure_occurrences
        self.frequency = frequency
        self.channels = channels

In [6]:
def healthy_patients(healthy_people_dict):
    healthy_patients = []
    for id, path in healthy_people_dict.items():
        samples = io.loadmat(path)['x']
        filename = os.path.basename(path)
        healthy_patients.append(Seizure(id, filename, samples))
 
    return healthy_patients

def unhealthy_patients(unhealthy_people_dict, seizure_occurrences):
    unhealthy_patients = []
    for id, path in unhealthy_people_dict.items():
        seizure_occ = seizure_occurrences[int(id)]
        samples = io.loadmat(path)['x']
        filename = os.path.basename(path)
        unhealthy_patients.append(Seizure(id, filename, samples, seizure_occurrences=seizure_occ))
 
    return unhealthy_patients

In [7]:
healthy_list: List[Seizure] = healthy_patients(healthy_patients_dict)
unhealthy_list: List[Seizure] = unhealthy_patients(unhealthy_patients_dict, seizure_occurrences)

In [8]:
# def saveToPng(figure, filename, img_dir, seizure_time):
#     base_filename = os.path.splitext(filename)[0]
#     dir_name = img_dir + base_filename
#     if not os.path.exists(dir_name):
#         os. makedirs(dir_name)
#     figure.savefig(dir_name + '/' + base_filename + str(seizure_time) + '.png')


# def plot_all_attacks(unhealthy_people, save_to_png=False, img_dir=""):
#  # colored blocks for visualization of attack
#     def create_annotations(occurrences):
#         events = [x for x in occurrences ]
#         return mne.Annotations(onset=events, duration=2, description=['seizure ' + str(x) for x in occurrences])

#     def visualize(filename, samples, seizure_time, all_events, save_to_png):
#         ch_names = list(map(str,range(1,17))) 
#         info = mne.create_info(ch_names=ch_names, sfreq =100)
#         raw = mne.io.RawArray(np.transpose(samples), info)
#         raw.set_annotations(all_events)

#         figure = raw.plot(n_channels=16, scalings='auto', title=filename + 'Time_of_seizure: ' + str(seizure_time), block=False, start=max(0, seizure_time - 5), duration=10)
 
#         if save_to_png:
#             saveToPng(figure, filename, img_dir, seizure_time)

#     for person in unhealthy_people:
#         all_events = create_annotations(person.seizure_occurrences)
#         for seizure_time in person.seizure_occurences:
#             visualize(person.filename, person.samples, seizure_time, all_events, save_to_png)
            
# def plot_healthy(healthy_people, save_to_png=False, img_dir=""):
#  # colored blocks for visualization of attack
#     def visualize(filename, samples, seizure_time, all_events, save_to_png):
#         ch_names = list(map(str,range(1,17))) 
#         info = mne.create_info(ch_names=ch_names, sfreq =100)
#         raw = mne.io.RawArray(np.transpose(samples), info)
#         raw.set_annotations(all_events)

#         figure = raw.plot(n_channels=16, scalings='auto', title=filename + 'Time_of_seizure: ' + str(seizure_time), block=False, start=max(0, seizure_time - 5), duration=10)
 
#         if save_to_png:
#             saveToPng(figure, filename, img_dir, seizure_time)

#     for person in unhealthy_people:
#         all_events = create_annotations(person.seizure_occurrences)
#         for seizure_time in person.seizure_occurences:
#             visualize(person.filename, person.samples, seizure_time, all_events, save_to_png)

In [9]:
# plot_all_attacks(unhealthy_list, save_to_png = True, img_dir=img_dir)
# list(map(str,range(1,17))) 

In [10]:
# def plot_seizures_of_patient(unhealthy_people, patient_nr, channels_no =1):
#     patient = next((x for x in unhealthy_people if x.id == str(patient_nr)), 0)
#     info = mne.create_info(ch_names = list(map(str,range(1,17))), sfreq = 100, ch_types ='eeg')
#     raw = mne.io.RawArray(np.transpose(patient.samples), info)
#     annot = mne.Annotations(onset = patient.seizure_matches, duration = 2, description = patient.seizure_matches) 
#     raw.set_annotations(annot)
#     plot_kwargs = {
#         'scalings': dict(eeg=30),
#         'highpass': 1,
#         'lowpass': 40,
#         'n_channels': channels_no,
#         'duration': 10,
#     }

#     for seizure in patient.seizure_occurences:
#         raw.plot(**plot_kwargs, start = seizure - 2)

In [11]:
for patient in healthy_list:
    patient.samples = np.delete(patient.samples, -1, axis=1)

In [12]:
seizure_time = 100

healthy_epochs = []
for healthy_patient in healthy_list:
    samples = healthy_patient.samples
    for i in range(0, samples.shape[0] - 100, seizure_time):
        chunk = samples[i:i + seizure_time]
        healthy_epochs.append(('healthy', chunk))


unhealthy_epochs = []
for unhealthy_patient in unhealthy_list:
    samples = unhealthy_patient.samples
    for seizure_time in unhealthy_patient.seizure_occurrences:
        seizure_start = seizure_time * 100
        chunk = samples[seizure_time:seizure_time + seizure_time]
        unhealthy_epochs.append(('unhealthy', chunk))


chunks_healthy_from_unhealthy = []
for unhealthy_patient in unhealthy_list:
    samples = unhealthy_patient.samples
    occurrences = unhealthy_patient.seizure_occurrences
    indices = [i * 100 for i in occurrences]

    # divide by seizure matches
    chunks = np.split(samples, indices)
    chunks_healthy = [chunks[0][: -(2*seizure_time)]]

     # remove 2 seconds of potential seizure from each chunk
    for element in chunks[1:]:
        chunks_healthy.append(element[(2*seizure_time):])

     # split each chunk for healthy waves
    for e in chunks_healthy:
        for i in range(0, e.shape [0] - 100, seizure_time):
            chunk = e[i:i + seizure_time]
            chunks_healthy_from_unhealthy.append(('healthy', chunk))            

In [13]:
def wavelet_decompose_channels(data, level):
#     data = data[0::2]
    data.columns.name ='channel'
    data_t = data.transpose()

    coeffs_list = wavedec(data_t.values, wavelet ='db4', level = level)

    nums = list(range(1, level + 1))
    names = []
    for num in nums:
        names.append('D' + str(num))
    names.append('A' + str(nums[ -1]))

    # reverse the names
    names = names[:: -1]
    wavelets = pd.DataFrame()

    for i, array in enumerate(coeffs_list):
        lvl_df = pd.DataFrame(array)
        lvl_df.index = data.columns
        lvl_df['level'] = names[i]
        lvl_df = lvl_df.set_index('level', append = True)
        lvl_df = lvl_df.T
        wavelets = pd.concat([ wavelets, lvl_df ], axis=1, sort = True)

    wavelets = wavelets.sort_values(['channel','level'], axis=1)

    to_be_dropped = [x for x in list(wavelets.columns.levels[1]) if not re.compile('D').match(x)]
    decom_wavelets = wavelets.drop(to_be_dropped, axis=1, level ='level')

    decom_wavelets.index.name ='sample'

    return decom_wavelets


class Feature:
    def __init__(self, max, min, mean, std, mean_abs) -> None:
        super().__init__()
        self.max = max
        self.min = min
        self.mean = mean
        self.std = std
        self.mean_abs = mean_abs

    def getall(self):
        import itertools
        return list(itertools.chain(self.max, self.min, self.mean, self.std, self.mean_abs))

all_label_features = []


def get_features(data):
    max_data = data.max().to_numpy()
    min_data = data.min().to_numpy()
    mean_data = data.mean().to_numpy()
    std_data = data.std().to_numpy()
    mean_abs_data = data.abs().mean().to_numpy()

    return Feature(max_data, min_data, mean_data, std_data, mean_abs_data)

def convert_y_to_number(str):
    return 1.0 if str == 'unhealthy' else 0.0

In [14]:
def model_1d_1(kernel_size, n_timesteps, n_features, n_outputs):
    model_1d_1 = keras.models.Sequential()
    model_1d_1.add(keras.layers.Conv1D(filters=64, kernel_size=kernel_size, strides=1, input_shape=(n_timesteps,n_features))) #
    model_1d_1.add(keras.layers.BatchNormalization())
    model_1d_1.add(keras.layers.LeakyReLU())
    model_1d_1.add(keras.layers.MaxPool1D(pool_size=2, strides=2))
    model_1d_1.add(keras.layers.Conv1D(filters=128, kernel_size=kernel_size, strides=1))
    model_1d_1.add(keras.layers.LeakyReLU())
    model_1d_1.add(keras.layers.MaxPool1D(pool_size=2, strides=2))
    model_1d_1.add(keras.layers.Dropout(0.5))
    model_1d_1.add(keras.layers.Conv1D(filters=256, kernel_size=kernel_size, strides=1))
    model_1d_1.add(keras.layers.LeakyReLU())
    model_1d_1.add(keras.layers.AveragePooling1D(pool_size=2, strides=2))
    model_1d_1.add(keras.layers.Dropout(0.5))
    model_1d_1.add(keras.layers.Conv1D(filters=512, kernel_size=kernel_size, strides=1))
    model_1d_1.add(keras.layers.LeakyReLU())
    model_1d_1.add(keras.layers.AveragePooling1D(pool_size=2, strides=2))
    model_1d_1.add(keras.layers.Conv1D(filters=1024, kernel_size=kernel_size, strides=1))
    model_1d_1.add(keras.layers.LeakyReLU())
    model_1d_1.add(keras.layers.GlobalAveragePooling1D())
    model_1d_1.add(keras.layers.Dense(n_outputs, activation="sigmoid"))
    model_1d_1.compile('sgd', loss='binary_crossentropy', metrics=['accuracy']) 
    return model_1d_1

def model_1d_1a(kernel_size, n_timesteps, n_features, n_outputs):
    model_1d_1a = keras.models.Sequential()
    model_1d_1a.add(keras.layers.Conv1D(filters=64, kernel_size=kernel_size, strides=1, input_shape=(n_timesteps,n_features))) #
    model_1d_1a.add(keras.layers.BatchNormalization())
    model_1d_1a.add(keras.layers.LeakyReLU())
    model_1d_1a.add(keras.layers.MaxPool1D(pool_size=2, strides=2))
    model_1d_1a.add(keras.layers.Conv1D(filters=128, kernel_size=kernel_size, strides=1))
    model_1d_1a.add(keras.layers.LeakyReLU())
    model_1d_1a.add(keras.layers.MaxPool1D(pool_size=2, strides=2))
    model_1d_1a.add(keras.layers.Dropout(0.5))
    model_1d_1a.add(keras.layers.Conv1D(filters=256, kernel_size=kernel_size, strides=1))
    model_1d_1a.add(keras.layers.LeakyReLU())
    model_1d_1a.add(keras.layers.AveragePooling1D(pool_size=2, strides=2))
    model_1d_1a.add(keras.layers.Dropout(0.5))
    model_1d_1a.add(keras.layers.Conv1D(filters=512, kernel_size=kernel_size, strides=1))
    model_1d_1a.add(keras.layers.LeakyReLU())
    model_1d_1a.add(keras.layers.AveragePooling1D(pool_size=2, strides=2))
    model_1d_1a.add(keras.layers.Conv1D(filters=1024, kernel_size=kernel_size, strides=1))
    model_1d_1a.add(keras.layers.LeakyReLU())
    model_1d_1a.add(keras.layers.GlobalAveragePooling1D())
    model_1d_1a.add(keras.layers.Dense(100, activation='relu'))
    model_1d_1a.add(keras.layers.Dense(n_outputs, activation="sigmoid"))
    model_1d_1a.compile('sgd', loss='binary_crossentropy', metrics=['accuracy'])
    return model_1d_1a


def model_1d_2(kernel_size, n_timesteps, n_features, n_outputs):
    model_1d_2 = keras.models.Sequential()
    model_1d_2.add(keras.layers.Conv1D(filters=5, kernel_size=kernel_size, strides=1, input_shape=(n_timesteps,n_features))) #
    model_1d_2.add(keras.layers.BatchNormalization())
    model_1d_2.add(keras.layers.LeakyReLU())
    model_1d_2.add(keras.layers.MaxPool1D(pool_size=2, strides=2))
    model_1d_2.add(keras.layers.Conv1D(filters=5, kernel_size=kernel_size, strides=1))
    model_1d_2.add(keras.layers.LeakyReLU())
    model_1d_2.add(keras.layers.MaxPool1D(pool_size=2, strides=2))
    model_1d_2.add(keras.layers.Dropout(0.5))
    model_1d_2.add(keras.layers.Conv1D(filters=5, kernel_size=kernel_size, strides=1))
    model_1d_2.add(keras.layers.LeakyReLU())
    model_1d_2.add(keras.layers.AveragePooling1D(pool_size=2, strides=2))
    model_1d_2.add(keras.layers.Dropout(0.5))
    model_1d_2.add(keras.layers.Conv1D(filters=5, kernel_size=kernel_size, strides=1))
    model_1d_2.add(keras.layers.LeakyReLU())
    model_1d_2.add(keras.layers.AveragePooling1D(pool_size=2, strides=2))
    model_1d_2.add(keras.layers.Conv1D(filters=5, kernel_size=kernel_size, strides=1))
    model_1d_2.add(keras.layers.LeakyReLU())
    model_1d_2.add(keras.layers.GlobalAveragePooling1D())
    model_1d_2.add(keras.layers.Dense(n_outputs, activation="sigmoid"))
    return model_1d_2


def model_1d_2a(kernel_size, n_timesteps, n_features, n_outputs):
    model_1d_2a = keras.models.Sequential()
    model_1d_2a.add(keras.layers.Conv1D(filters=5, kernel_size=kernel_size, strides=1, input_shape=(n_timesteps,n_features))) #
    model_1d_2a.add(keras.layers.BatchNormalization())
    model_1d_2a.add(keras.layers.LeakyReLU())
    model_1d_2a.add(keras.layers.MaxPool1D(pool_size=2, strides=2))
    model_1d_2a.add(keras.layers.Conv1D(filters=5, kernel_size=kernel_size, strides=1))
    model_1d_2a.add(keras.layers.LeakyReLU())
    model_1d_2a.add(keras.layers.MaxPool1D(pool_size=2, strides=2))
    model_1d_2a.add(keras.layers.Dropout(0.5))
    model_1d_2a.add(keras.layers.Conv1D(filters=5, kernel_size=kernel_size, strides=1))
    model_1d_2a.add(keras.layers.LeakyReLU())
    model_1d_2a.add(keras.layers.AveragePooling1D(pool_size=2, strides=2))
    model_1d_2a.add(keras.layers.Dropout(0.5))
    model_1d_2a.add(keras.layers.Conv1D(filters=5, kernel_size=kernel_size, strides=1))
    model_1d_2a.add(keras.layers.LeakyReLU())
    model_1d_2a.add(keras.layers.AveragePooling1D(pool_size=2, strides=2))
    model_1d_2a.add(keras.layers.Conv1D(filters=5, kernel_size=kernel_size, strides=1))
    model_1d_2a.add(keras.layers.LeakyReLU())
    model_1d_2a.add(keras.layers.GlobalAveragePooling1D())
    model_1d_2a.add(keras.layers.Dense(100, activation='relu'))
    model_1d_2a.add(keras.layers.Dense(n_outputs, activation="sigmoid"))
    return model_1d_2a


def model_1d_3(kernel_size, n_timesteps, n_features, n_outputs):
    model_1d_3 = keras.models.Sequential()
    model_1d_3.add(keras.layers.Conv1D(filters=64, kernel_size=kernel_size, activation='relu', input_shape=(n_timesteps,n_features)))
    model_1d_3.add(keras.layers.Conv1D(filters=64, kernel_size=kernel_size, activation='relu'))
    model_1d_3.add(keras.layers.Dropout(0.5))
    model_1d_3.add(keras.layers.MaxPooling1D(pool_size=2))
    model_1d_3.add(keras.layers.Flatten())
    model_1d_3.add(keras.layers.Dense(100, activation='relu'))
    model_1d_3.add(keras.layers.Dense(n_outputs, activation='sigmoid'))
    return model_1d_3





In [15]:
class TestResult:
    def __init__ (self, id, batch_size, n_epochs, kernel_size, model_summary, history, chgw, accuracy, elapsed_transforming_time, elapsed_training_time) -> None :
        super (). __init__ ()
        self.id = id
        self.batch_size = batch_size
        self.n_epochs = n_epochs
        self.kernel_size = kernel_size
        self.model_summary = model_summary
        self.history = history
        self.chgw = chgw
        self.accuracy = accuracy
        self.elapsed_transforming_time = elapsed_transforming_time
        self.elapsed_training_time = elapsed_training_time

    

In [16]:
time_identifier = datetime.today().strftime('%Y%m%d%H%M%S')
r_state = np.random.RandomState()

models_avaluation = []

data = {}
# data["healthy_size"] = [1000, 2000, 5000, 10000, 25000, 68224]
data["healthy_size"] = [2500]
# data["healthy_from_unhealthy_size"] = [0, 1000, 2000, 5000, 10993]
# data["models"] = [model_1d_1, model_1d_1a, model_1d_2, model_1d_2a, model_1d_3]
data["models"] = [model_1d_1, model_1d_1a]
data["kernel_size"] = [3, 5, 7, 11]
data["n_epochs"] = [10, 20, 30]
data["batch_size"] = [32, 64]
verbose = 1

for f_model in data["models"]:
    for healthy_size in data["healthy_size"]:
        print("Processed size: %i" % healthy_size)
        start_transforming = time.time()
        
        
        all_label_features = []
        healthy_from_unhealthy_size = healthy_size * ((len(chunks_healthy_from_unhealthy) / len(healthy_epochs)))
        true_healthy = random.sample(healthy_epochs, healthy_size)
        healthy_chunks_from_unhealthy = random.sample(chunks_healthy_from_unhealthy, int(healthy_from_unhealthy_size))
        all_epochs = true_healthy + unhealthy_epochs + healthy_chunks_from_unhealthy

        for epoch in all_epochs:
            wavelet = wavelet_decompose_channels(pd.DataFrame(epoch[1]), level=4)
            features = get_features(wavelet)
            all_label_features.append((epoch[0], features))


        X = np.array(list(x[1].getall() for x in all_label_features)) # data
        y = np.array(list(convert_y_to_number(x[0]) for x in all_label_features)) # labels

        X_r = np.reshape(X, (X.shape[0], X.shape[1], 1))
        y_r = np.reshape(y, (y.shape[0], 1))


        X_train , X_test , y_train , y_test = train_test_split(X_r, y_r, test_size=0.4, random_state=r_state, shuffle=True)
        n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
        end_transforming = time.time()
        elapsed_transforming_time = end_transforming - start_transforming
        print("Elapsed time for processing: %s" % str(elapsed_transforming_time))
        print("Start evaluating the model parameters...")
        start_evaluating = time.time()
        
        l_n_timesteps = [n_timesteps]
        l_n_features = [n_features]
        l_n_outputs = [n_outputs]

        model = KerasClassifier(build_fn=f_model, verbose=0)
        param_grid = dict(batch_size=data["batch_size"], epochs=data["n_epochs"], kernel_size=data["kernel_size"], n_timesteps=l_n_timesteps, n_features=l_n_features, n_outputs=l_n_outputs)
        grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
        grid_result = grid.fit(X_r, y_r)
        # summarize results
        print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
        means = grid_result.cv_results_['mean_test_score']
        stds = grid_result.cv_results_['std_test_score']
        params = grid_result.cv_results_['params']
        for mean, stdev, param in zip(means, stds, params):
            print("%f (%f) with: %r" % (mean, stdev, param))
            
        end_evaluating = time.time()
        elapsed_evaluating_time = end_evaluating - start_evaluating
        
#         for kernel_size in data["kernel_size"]:
#             for n_epochs in data["n_epochs"]:
#                 for batch_size in data["batch_size"]:
#                     identifier = f'model_%i_%i_%i_kernel_%i_epochs_%s_batch_%i' % (model_id, healthy_from_unhealthy_size, healthy_size, kernel_size, n_epochs, batch_size)

#                     start_training = time.time()

#                     model = f_model(kernel_size, n_timesteps, n_features, n_outputs)

#                     model.summary()



#                     history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_split=0.1)
#                     _, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)

#                     if not os.path.exists(plots_path + '/' + time_identifier):
#                         os.makedirs(plots_path + '/' + time_identifier)


#                     pd.DataFrame(history.history).plot(figsize=(8, 10))
#                     plt.grid(True)
#                     plt.gca().set_ylim(0, 1)
#                     plt.savefig(plots_path + '/' + time_identifier + '/plot_' + identifier + '.png')

#                     end_training = time.time()
#                     elapsed_training_time = end_training - start_training
#                     models_avaluation.append(TestResult(id=identifier, kernel_size=kernel_size, n_epochs=n_epochs, batch_size=batch_size, model_summary=model.summary(), history=history, chgw=_, accuracy=accuracy, elapsed_transforming_time=elapsed_transforming_time, elapsed_training_time=elapsed_training_time))



Processed size: 2500


/home/dominik_zawrotny/ml/ml_venv/lib/python3.7/site-packages/pywt/_multilevel.py:45: UserWarning: Level value of 4 is too high: all coefficients will experience boundary effects.
  "boundary effects.").format(level))


Elapsed time for processing: 64.73258781433105
Start evaluating the model parameters...


/home/dominik_zawrotny/ml/ml_venv/lib/python3.7/site-packages/ipykernel_launcher.py:54: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
2022-05-22 22:03:49.441965: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-05-22 22:03:49.441971: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-05-22 22:03:49.447349: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda

2022-05-22 22:03:51.827422: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-22 22:03:51.827945: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-22 22:03:51.828064: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ml-vm): /proc/driver/nvidia/version does not exist
2022-05-22 22:03:51.828415: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-22 22:03:51.852146: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could

2022-05-22 22:48:08.772506: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-05-22 22:48:08.784739: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-22 22:48:08.786182: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-05-22 22:48:19.224087: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-22 22:48:19.224674: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to c

Best: 0.835838 using {'batch_size': 64, 'epochs': 10, 'kernel_size': 7, 'n_features': 1, 'n_outputs': 1, 'n_timesteps': 320}
0.835560 (0.230784) with: {'batch_size': 32, 'epochs': 10, 'kernel_size': 3, 'n_features': 1, 'n_outputs': 1, 'n_timesteps': 320}
0.835559 (0.230786) with: {'batch_size': 32, 'epochs': 10, 'kernel_size': 5, 'n_features': 1, 'n_outputs': 1, 'n_timesteps': 320}
0.835281 (0.230587) with: {'batch_size': 32, 'epochs': 10, 'kernel_size': 7, 'n_features': 1, 'n_outputs': 1, 'n_timesteps': 320}
nan (nan) with: {'batch_size': 32, 'epochs': 10, 'kernel_size': 11, 'n_features': 1, 'n_outputs': 1, 'n_timesteps': 320}
0.835003 (0.230393) with: {'batch_size': 32, 'epochs': 20, 'kernel_size': 3, 'n_features': 1, 'n_outputs': 1, 'n_timesteps': 320}
0.835281 (0.230587) with: {'batch_size': 32, 'epochs': 20, 'kernel_size': 5, 'n_features': 1, 'n_outputs': 1, 'n_timesteps': 320}
0.833891 (0.229606) with: {'batch_size': 32, 'epochs': 20, 'kernel_size': 7, 'n_features': 1, 'n_outputs

2022-05-22 23:01:57.370457: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-05-22 23:01:57.376291: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-22 23:01:57.376385: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-05-22 23:01:57.425768: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-05-22 23:01:57.4311

2022-05-22 23:37:34.416735: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-22 23:37:34.417223: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-22 23:37:34.417314: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ml-vm): /proc/driver/nvidia/version does not exist
2022-05-22 23:37:34.417699: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-22 23:38:03.581280: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. 

Best: 0.836394 using {'batch_size': 64, 'epochs': 10, 'kernel_size': 7, 'n_features': 1, 'n_outputs': 1, 'n_timesteps': 320}
0.833335 (0.229211) with: {'batch_size': 32, 'epochs': 10, 'kernel_size': 3, 'n_features': 1, 'n_outputs': 1, 'n_timesteps': 320}
0.835004 (0.230397) with: {'batch_size': 32, 'epochs': 10, 'kernel_size': 5, 'n_features': 1, 'n_outputs': 1, 'n_timesteps': 320}
0.832500 (0.228621) with: {'batch_size': 32, 'epochs': 10, 'kernel_size': 7, 'n_features': 1, 'n_outputs': 1, 'n_timesteps': 320}
nan (nan) with: {'batch_size': 32, 'epochs': 10, 'kernel_size': 11, 'n_features': 1, 'n_outputs': 1, 'n_timesteps': 320}
0.834169 (0.229801) with: {'batch_size': 32, 'epochs': 20, 'kernel_size': 3, 'n_features': 1, 'n_outputs': 1, 'n_timesteps': 320}
0.834447 (0.229998) with: {'batch_size': 32, 'epochs': 20, 'kernel_size': 5, 'n_features': 1, 'n_outputs': 1, 'n_timesteps': 320}
0.832778 (0.228829) with: {'batch_size': 32, 'epochs': 20, 'kernel_size': 7, 'n_features': 1, 'n_outputs

In [17]:
time_identifier = datetime.today().strftime('%Y%m%d%H%M%S')
r_state = np.random.RandomState()

models_avaluation = []

data = {}
data["healthy_size"] = [1000, 2000, 5000, 10000, 25000, 68224]
data["models"] = [model_1d_1, model_1d_1a]
data["kernel_size"] = [2, 3, 5, 7]
data["n_epochs"] = [10, 20, 30]
data["batch_size"] = [32, 64]
verbose = 1

gd_path = 

for model_id, f_model in enumerate(data["models"]):
    for healthy_size in data["healthy_size"]:
        identifier = f'model_%i_%i' % (model_id, healthy_size)
        print("Processed size: %i" % healthy_size)
        start_transforming = time.time()
        
        
        all_label_features = []
        healthy_from_unhealthy_size = healthy_size * ((len(chunks_healthy_from_unhealthy) / len(healthy_epochs)))
        true_healthy = random.sample(healthy_epochs, healthy_size)
        healthy_chunks_from_unhealthy = random.sample(chunks_healthy_from_unhealthy, int(healthy_from_unhealthy_size))
        all_epochs = true_healthy + unhealthy_epochs + healthy_chunks_from_unhealthy

        for epoch in all_epochs:
            wavelet = wavelet_decompose_channels(pd.DataFrame(epoch[1]), level=4)
            features = get_features(wavelet)
            all_label_features.append((epoch[0], features))


        X = np.array(list(x[1].getall() for x in all_label_features)) # data
        y = np.array(list(convert_y_to_number(x[0]) for x in all_label_features)) # labels

        X_r = np.reshape(X, (X.shape[0], X.shape[1], 1))
        y_r = np.reshape(y, (y.shape[0], 1))


        X_train , X_test , y_train , y_test = train_test_split(X_r, y_r, test_size=0.2, random_state=r_state, shuffle=True)
        n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
        end_transforming = time.time()
        elapsed_transforming_time = end_transforming - start_transforming
        print("Elapsed time for processing: %s" % str(elapsed_transforming_time))
        print("Start evaluating the model parameters...")
        start_evaluating = time.time()
        
        l_n_timesteps = [n_timesteps]
        l_n_features = [n_features]
        l_n_outputs = [n_outputs]

        model = KerasClassifier(build_fn=f_model, verbose=0)
        param_grid = dict(batch_size=data["batch_size"], epochs=data["n_epochs"], kernel_size=data["kernel_size"], n_timesteps=l_n_timesteps, n_features=l_n_features, n_outputs=l_n_outputs)
        grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
        grid_result = grid.fit(X_r, y_r)
        # summarize results
            
        if not os.path.exists(gd_path + '/' + time_identifier):
            os.makedirs(gd_path + '/' + time_identifier)

        result_f = open(gd_path + '/' + time_identifier + '/result_' + identifier + '.txt', "w")
        result_f.write("Processed size: %i" % healthy_size)
        result_f.write("\n\n")
        
        print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
        result_f.write("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
        result_f.write("\n\n\n")
        
        means = grid_result.cv_results_['mean_test_score']
        stds = grid_result.cv_results_['std_test_score']
        params = grid_result.cv_results_['params']
        for mean, stdev, param in zip(means, stds, params):
            print("%f (%f) with: %r" % (mean, stdev, param))
            result_f.write("%f (%f) with: %r" % (mean, stdev, param))
            result_f.write("\n")
            
            
        end_evaluating = time.time()
        elapsed_evaluating_time = end_evaluating - start_evaluating
        result_f.write("Evaluating time: %s" % str(elapsed_evaluating_time))
        result_f.write("\n\n")
        result_f.write("Transforming time: %s" % str(elapsed_transforming_time))
        result_f.write("\n\n")
        result_f..close()
        
#         for kernel_size in data["kernel_size"]:
#             for n_epochs in data["n_epochs"]:
#                 for batch_size in data["batch_size"]:
#                     identifier = f'model_%i_%i_%i_kernel_%i_epochs_%s_batch_%i' % (model_id, healthy_from_unhealthy_size, healthy_size, kernel_size, n_epochs, batch_size)

#                     start_training = time.time()

#                     model = f_model(kernel_size, n_timesteps, n_features, n_outputs)

#                     model.summary()



#                     history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_split=0.1)
#                     _, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)

#                     if not os.path.exists(plots_path + '/' + time_identifier):
#                         os.makedirs(plots_path + '/' + time_identifier)


#                     pd.DataFrame(history.history).plot(figsize=(8, 10))
#                     plt.grid(True)
#                     plt.gca().set_ylim(0, 1)
#                     plt.savefig(plots_path + '/' + time_identifier + '/plot_' + identifier + '.png')

#                     end_training = time.time()
#                     elapsed_training_time = end_training - start_training
#                     models_avaluation.append(TestResult(id=identifier, kernel_size=kernel_size, n_epochs=n_epochs, batch_size=batch_size, model_summary=model.summary(), history=history, chgw=_, accuracy=accuracy, elapsed_transforming_time=elapsed_transforming_time, elapsed_training_time=elapsed_training_time))



Processed size: 5000
Elapsed time for processing: 119.97853565216064
Start evaluating the model parameters...


/home/dominik_zawrotny/ml/ml_venv/lib/python3.7/site-packages/ipykernel_launcher.py:54: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
2022-05-23 00:01:06.124247: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-05-23 00:01:06.124257: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-05-23 00:01:06.129741: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda

2022-05-23 00:59:03.647965: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-05-23 00:59:03.683384: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-23 00:59:03.683637: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-05-23 00:59:13.578304: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-23 00:59:13.578882: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to c

2022-05-23 01:11:13.015596: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-23 01:11:13.016181: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-23 01:11:13.016224: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ml-vm): /proc/driver/nvidia/version does not exist
2022-05-23 01:11:13.016723: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-23 01:27:52.681715: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. 

Best: 0.910495 using {'batch_size': 32, 'epochs': 30, 'kernel_size': 7, 'n_features': 1, 'n_outputs': 1, 'n_timesteps': 320}
0.908526 (0.124250) with: {'batch_size': 32, 'epochs': 10, 'kernel_size': 3, 'n_features': 1, 'n_outputs': 1, 'n_timesteps': 320}
0.908224 (0.124014) with: {'batch_size': 32, 'epochs': 10, 'kernel_size': 5, 'n_features': 1, 'n_outputs': 1, 'n_timesteps': 320}
0.909284 (0.124758) with: {'batch_size': 32, 'epochs': 10, 'kernel_size': 7, 'n_features': 1, 'n_outputs': 1, 'n_timesteps': 320}
nan (nan) with: {'batch_size': 32, 'epochs': 10, 'kernel_size': 11, 'n_features': 1, 'n_outputs': 1, 'n_timesteps': 320}
0.907466 (0.123497) with: {'batch_size': 32, 'epochs': 20, 'kernel_size': 3, 'n_features': 1, 'n_outputs': 1, 'n_timesteps': 320}
0.908072 (0.123925) with: {'batch_size': 32, 'epochs': 20, 'kernel_size': 5, 'n_features': 1, 'n_outputs': 1, 'n_timesteps': 320}
0.907921 (0.123840) with: {'batch_size': 32, 'epochs': 20, 'kernel_size': 7, 'n_features': 1, 'n_outputs

...

2022-05-23 14:38:56.665185: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-05-23 14:38:56.715923: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-23 14:38:56.715982: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-05-23 14:39:06.968585: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-23 14:39:06.978656: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to c

Best: 0.910798 using {'batch_size': 64, 'epochs': 30, 'kernel_size': 7, 'n_features': 1, 'n_outputs': 1, 'n_timesteps': 320}
0.910041 (0.125296) with: {'batch_size': 32, 'epochs': 10, 'kernel_size': 3, 'n_features': 1, 'n_outputs': 1, 'n_timesteps': 320}
0.909284 (0.124760) with: {'batch_size': 32, 'epochs': 10, 'kernel_size': 5, 'n_features': 1, 'n_outputs': 1, 'n_timesteps': 320}
0.910495 (0.125616) with: {'batch_size': 32, 'epochs': 10, 'kernel_size': 7, 'n_features': 1, 'n_outputs': 1, 'n_timesteps': 320}
nan (nan) with: {'batch_size': 32, 'epochs': 10, 'kernel_size': 11, 'n_features': 1, 'n_outputs': 1, 'n_timesteps': 320}
0.910344 (0.125510) with: {'batch_size': 32, 'epochs': 20, 'kernel_size': 3, 'n_features': 1, 'n_outputs': 1, 'n_timesteps': 320}
0.908829 (0.124464) with: {'batch_size': 32, 'epochs': 20, 'kernel_size': 5, 'n_features': 1, 'n_outputs': 1, 'n_timesteps': 320}
0.910344 (0.125510) with: {'batch_size': 32, 'epochs': 20, 'kernel_size': 7, 'n_features': 1, 'n_outputs

2022-05-23 15:26:19.362348: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-05-23 15:26:19.368434: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-23 15:26:19.368532: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-05-23 15:26:19.531686: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-05-23 15:26:19.5316

2022-05-23 15:26:21.827936: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-23 15:26:21.828341: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-23 15:26:21.828419: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ml-vm): /proc/driver/nvidia/version does not exist
2022-05-23 15:26:21.828742: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-23 15:26:21.896545: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could

KeyboardInterrupt: 

In [22]:
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV

def search_for_best_parameter(classifier, parameters, scoring):
    clf = GridSearchCV(classifier, parameters, n_jobs=-1, cv=3, scoring=scoring)
    clf.fit(X_train, y_train)

    print('Best parameters found :\n', clf.best_params_)

scores = ['balanced_accuracy', 'precision', 'recall', 'f1']

parameter_space = { 
    'n_neighbors': range(1,20),
    'leaf_size': range(5, 100, 5)
}

for score in scores :
    search_for_best_parameter(KNeighborsClassifier(), parameter_space, score)



Best parameters found :
 {'leaf_size': 5, 'n_neighbors': 1}
Best parameters found :
 {'leaf_size': 5, 'n_neighbors': 2}
Best parameters found :
 {'leaf_size': 5, 'n_neighbors': 1}
Best parameters found :
 {'leaf_size': 5, 'n_neighbors': 1}


In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier


knn_class = [
    ("KNN (1 nn, 70 leaf size)", KNeighborsClassifier(1, leaf_size=70)),
    ("KNN (2 nn, 30 leaf size)", KNeighborsClassifier(2, leaf_size=30)),
    ("KNN (14 nn, 30 leaf size)", KNeighborsClassifier(14, leaf_size=30)),
]

svm_class = [
    ("SVM (rbf1) ", SVC(random_state=r_state, kernel="rbf", C=100, degree=5, probability=True)),
    ("SVM (rbf2)", SVC(random_state=r_state, kernel="rbf", C=1, degree=2, probability=True)),
    ("SVM (rbf3)", SVC(random_state=r_state, kernel="rbf", C=10, degree=2, probability=True)),
]

# mlp_class = [
#     ("MLP1", MLPClassifier(max_iter=150, learning_rate='adaptive', random_state=r_state, random_state=(150, 100, 50), activation='logistic', solver='adam')),
#     ("MLP2", MLPClassifier(max_iter=150, learning_rate='adaptive', random_state=r_state, random_state=(120, 80, 40), activation='relu', solver='lbfgs')),
#     ("MLP3", MLPClassifier(max_iter=100, learning_rate='adaptive', random_state=r_state, random_state=(120, 80, 40), activation='tanh', solver='lbfgs'))
# ]

dtree_class = [
    ("DECISION TREE (max depth = 1)", DecisionTreeClassifier(max_depth=1, random_state=r_state)),
    ("DECISION TREE (max depth = 2)", DecisionTreeClassifier(max_depth=2, random_state=r_state)),
    ("DECISION TREE (max depth = 4)", DecisionTreeClassifier(max_depth=4, random_state=r_state)),
    ("DECISION TREE (max depth = 6)", DecisionTreeClassifier(max_depth=6, random_state=r_state)),
    ("DECISION TREE (max depth = 8)", DecisionTreeClassifier(max_depth=8, random_state=r_state)),
    ("DECISION TREE (max depth = 10)", DecisionTreeClassifier(max_depth=10, random_state=r_state)),
    ("DECISION TREE (max depth = 12)", DecisionTreeClassifier(max_depth=12, random_state=r_state)),
]

random_forest_classifier = RandomForestClassifier(random_state=r_state, n_estimators=140)

classifiers = [
    ("Nearest Neighbors", knn_class),
    ("Linear SVM", svm_class),
    ("Decision Tree", dtree_class),
    ("Random Forest", random_forest_classifier),
#     ("Multilayer Perceptron", mlp_vclass)
]

eclf = VotingClassifier(estimators = classifiers, voting = 'soft', weights = [3, 2, 1, 1])


def score_for_vclassifiers(classificator, name):
    vclass = VotingClassifier(estimators=classificator, voting='soft')
    vclass.fit(X_train, y_train)
    score = vclass.score(X_test, y_test)
    print(f"VotingClassifier for %s: %.2f%%"%(name, (score * 100))) 
    return vclass


def savePlot(figure, class_name, metric_type):
    figure.savefig(dir_measures + '/' + class_name + '-' + metric_type +'.png')


def score_evaluation(classificator,  classificator_name, save_metrics = False):
    print(f"SCORE FOR %s\n"%  classificator_name)

    y_pred_prob = classificator.predict_proba(X_test)

    # ROC CURVE

    roc_plot = skplt.metrics.plot_roc_curve(y_test, y_pred_prob)

    # PRECISION RECALL
    prec_recall_plot = skplt.metrics.plot_precision_recall(y_test, y_pred_prob)

    print("CONFUSION MATRIX ")
    y_pred_normal = classificator.predict(X_test)

    conf_matrix_plot = skplt.metrics.plot_confusion_matrix(y_test, y_pred_normal, labels =[1, 0])

    conf_matrix = confusion_matrix(y_test, y_pred_normal, labels =[1, 0])

    print(conf_matrix)

    tp = conf_matrix [0][0]
    fp = conf_matrix [0][1]
    fn = conf_matrix [1][0]
    tn = conf_matrix [1][1]
    print("\ nTrue positive: ", tp)
    print("False positive: ", fp)
    print("False negative: ", fn)
    print("True negaitve: ", tn)

    # ACC
    from sklearn.metrics import accuracy_score
    acc_score = accuracy_score(y_test, y_pred_normal)

    acc_str = f"Accuracy : %.2 f%%"%(acc_score * 100)
    print('\n', acc_str)

    from sklearn.metrics import f1_score
    fscore = f1_score(y_test, y_pred_normal, average = None)

    f1score_str = "F1 score [ NOT SEIZURE, SEIZURE ] "+ str(fscore)
    print(f1score_str)

    # SENSITIVITY
    def safe_div(n, d):
        return n / d if d else 0

    sensitivity = safe_div(tp, (tp + fn))
    sensitivity_str = f"Sensitivity (true positive rate): %.2 f%%"%(
    sensitivity * 100)
    print(sensitivity_str)

    # SPECIFITY
    specifity = safe_div(tn, (tn + fp))
    specifity_str = f"Specificity (true negative rate): %.2f %%\ n"%(
    specifity * 100)
    print(specifity_str)

    # PRECISION
    precision = safe_div(tp, (tp + fp))
    precision_str = f"Precision: %.2 f%%\ n"%(
    precision * 100)
    print(precision_str)

    if sav_metrics:
        savePlot(roc_plot.get_figure(),  classificator_name, "ROC ")
        savePlot(prec_recall_plot.get_figure(),  classificator_name, "PRECISIONRECALL ")
        savePlot(conf_matrix_plot.get_figure(),  classificator_name, "CONFUSIONMATRIX ")
        lines = [ acc_str, f1score_str, sensitivity_str, specifity_str, precision_str ]
        with open(dir_measures + sep +  classificator_name +'-metrics.txt', 'w') as f:
            f.write('\n'.join(lines))